# Affichage des données téléphoniques

## Imports

In [ ]:
import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import folium
import pandas as pd;
import branca
from copy import deepcopy
from selenium import webdriver
from PIL import Image
import io
from selenium.webdriver.chrome.service import Service
from pathlib import Path
from PIL import Image
import os
from pyproj import Transformer
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.cluster import HDBSCAN
import numpy as np

from useful_methods.ihm.maps.mapUtils import *
from useful_methods.neighbours_delaunay.simple_criteria import *
from useful_methods.neighbours_delaunay.enhanced_criteria import *
from useful_methods.data_processing import extract_data
from useful_methods.neighbours_delaunay.graphs import delaunay_graph


In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')
providers = df['nom_op'].unique()
regions = df['nom_reg'].unique()    
technologies = pd.Series(['site_2g', 'site_3g', 'site_4g', 'site_5g'])
ntechs = len(technologies)
nprovs = len(providers)

## Plotting and saving subplots of maps

Fonction pour créer une image à partir d'un fichier html 

In [ ]:
def save_png(html_file, png_file, compressing_factor = 1):      
    # Configurer Selenium WebDriver
    service = Service(executable_path=r'/usr/bin/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(2760, 1600)
    # Charger la page HTML dans le navigateur
    driver.get(f"file://{str(Path().absolute())}/{html_file}")

    # Prendre une capture d'écran de l'élément
    screenshot = driver.get_screenshot_as_png()
    driver.quit()

    # Convertir la capture d'écran en image avec Pillow
    image = Image.open(io.BytesIO(screenshot))

    # Compresser l'image
    width, height = image.size
    new_size = (width//compressing_factor, height//compressing_factor)
    image = image.resize(new_size)

    # Enregistrer l'image en PNG
    image.save(png_file)

Traçage des comparaisons des différents opérateurs pour chaque technologie

In [ ]:
for technology in technologies :
    fig = branca.element.Figure()
    for i, provider in enumerate(providers) :
        subplot = fig.add_subplot(int(nprovs/2), nprovs-int(nprovs/2), i+1)
        subplot.add_child(getMap(deepcopy(df), [provider], regions, [technology], "Opérateurs", location = [46.8, 2.3], zoom_start = 6)) 
    addLegend(fig,{'Bouygues Telecom':'#035afc', 'Orange':'#fc5603', 'SFR':'#169e26', 'Free Mobile':'#dbd640'})
    fig.save(f"../../out/maps/providers-{technology}.html")
    save_png(f"../../out/maps/providers-{technology}.html", f"../../out/images/providers-{technology}.png", 2)

Traçage des répartitions des stations de base selon les opérateurs

In [ ]:
fig = branca.element.Figure()
for i,provider in enumerate(providers) :
    subplot = fig.add_subplot(int(nprovs/2), nprovs-int(nprovs/2), i+1)
    subplot.add_child(getMap(deepcopy(df), [provider], regions, technologies, "Opérateurs", location = [46.8, 2.3], zoom_start = 6)) 
addLegend(fig,{'Bouygues Telecom':'#035afc', 'Orange':'#fc5603', 'SFR':'#169e26', 'Free Mobile':'#dbd640'})
fig.save("../../out/maps/subplots-operators.html")
save_png("../../out/maps/subplots-operators.html", "../../out/images/subplots-operators.png", 2)

### Couverture ciblée

In [ ]:
data_couv_cible = pd.read_csv("../../database/2023_T4_dcc_Metropole_sites.csv", sep=";", decimal=',')
Nan_values_data_couv_cible = data_couv_cible[data_couv_cible['x_lambert_93'].isnull() | data_couv_cible['y_lambert_93'].isnull()]
data_couv_cible.dropna(axis=0, how='any', subset=['x_lambert_93', 'y_lambert_93'], inplace=True)


In [ ]:
Nan_values_data_couv_cible[['x_lambert_93', 'y_lambert_93']]

In [ ]:
map = folium.Map(location=[46.8, 2.3], zoom_start=7, tiles="Cartodb Positron")

# Définir le transformateur pour la conversion de Lambert 93 à WGS84
transformer = Transformer.from_crs("epsg:2154", "epsg:4326")

for ind, x, y in data_couv_cible[['x_lambert_93', 'y_lambert_93']].itertuples():
    # Convertir les coordonnées Lambert 93 en longitude/latitude
    latitude, longitude = transformer.transform(x, y)
    map.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, popup=f"({x}, {y})"))
map.save("../../out/maps/couverture_ciblee.html")
save_png("../../out/maps/couverture_ciblee.html","../../out/images/couverture_ciblee.png", 2)

## H-DBScan

In [ ]:
def probaCity(coordsXY):
    clusterer= HDBSCAN(min_cluster_size=5, min_samples=40)
    clusterer.fit(coordsXY)
    return pd.Series(data = clusterer.probabilities_, index=coordsXY.index)

In [ ]:
df_filtered = df[df['nom_op']=='Orange']
positions = df_filtered[['x','y']]
positionsLl = df_filtered[['latitude', 'longitude']]

In [ ]:
probabilities = probaCity(positions)

In [ ]:
# Create a colormap
cmap = plt.cm.get_cmap('winter_r')  # 'cool' colormap for blue to gray
norm = mcolors.Normalize(vmin=0, vmax=1)

# Create a map centered on France
map = folium.Map(location=[46.8, 2.3], zoom_start=7, tiles="Cartodb Positron")

for ind, longitude, latitude in positionsLl.itertuples():
    proba = probabilities[ind]
    color = 'gray' if proba == 0 else mcolors.to_hex(cmap(norm(proba)))
    map.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], color=color, radius=1.5, popup=f"proba={probabilities[ind]}"))
map.save("../../out/maps/villes_HDBSCAN.html")
save_png("../../out/maps/villes_HDBSCAN.html","../../out/images/villes_HDBSCAN.png", 2)

## Afficher le graphe de voisinage

In [ ]:
# Create a map centered on France
import folium.features


map = folium.Map(location=[46.8, 2.3], zoom_start=7, tiles="Cartodb Positron")
labels = folium.FeatureGroup("Labels").add_to(map)
points = folium.FeatureGroup("Points").add_to(map)
#Create graph
df_extracted = extract_data(df, provider='Orange', techno='4g', region='Normandie')
G, pos = delaunay_graph(df_extracted)

G = distance_criterion(G, pos)
G = angle_criterion(G, pos)
G = quadrant_criterion(G, pos)

for edge in G.edges:
    stations = []
    stations.append(pos[edge[0]])
    stations.append(pos[edge[1]])
    stations = np.array(stations)
    
    folium.PolyLine(stations, color="red", weight=2.5, opacity=1).add_to(map)
    folium.map.Marker((stations[0]+stations[1])/2, icon=folium.features.DivIcon(icon_size=(250,36), icon_anchor=(0,0),html=f'<div style="font-size: 7pt">{km_distance(stations[0], stations[1]):.2f} km</div>')
    ).add_to(labels)
for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    points.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], color='blue', radius=2.5))
folium.LayerControl().add_to(map)

map.save("../../out/maps/neighborhood_default.html")


In [ ]:
# Create a map centered on France
import folium.features


map = folium.Map(location=[46.8, 2.3], zoom_start=7, tiles="Cartodb Positron")
labels = folium.FeatureGroup("Labels").add_to(map)
points = folium.FeatureGroup("Points").add_to(map)
#Create graph
df_extracted = extract_data(df, provider='Orange', techno='4g', region='Normandie')
G, pos = delaunay_graph(df_extracted)

cityness_proba = probaCity(df_extracted[['x', 'y']])
G = distance_criterion_enhanced(G, pos, cityness_proba = cityness_proba)
G = angle_criterion_enhanced(G, pos, cityness_proba = cityness_proba)
G = quadrant_criterion_enhanced(G, pos)

for edge in G.edges:
    stations = []
    stations.append(pos[edge[0]])
    stations.append(pos[edge[1]])
    stations = np.array(stations)
    
    folium.PolyLine(stations, color="red", weight=2.5, opacity=1).add_to(map)
    folium.map.Marker((stations[0]+stations[1])/2, icon=folium.features.DivIcon(icon_size=(250,36), icon_anchor=(0,0),html=f'<div style="font-size: 7pt">{km_distance(stations[0], stations[1]):.2f} km</div>')
    ).add_to(labels)
for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    points.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], color='blue', radius=2.5, popup=f"proba={cityness_proba.get(ind)}"))
folium.LayerControl().add_to(map)

map.save("../../out/maps/enhanced_neighborhood_default.html")
